# Gradient Decode Dicom Tensorflow Operation Example

In [1]:
!apt-get install libdcmtk-dev
!pip3 install gradient-decode-dicom

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcharls-dev libdcmtk12 libwrap0-dev
Suggested packages:
  dcmtk-doc
The following NEW packages will be installed:
  libcharls-dev libdcmtk-dev libdcmtk12 libwrap0-dev
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,445 kB of archives.
After this operation, 35.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcharls-dev amd64 1.1.0+dfsg-2 [20.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libwrap0-dev amd64 7.6.q-27 [21.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libdcmtk12 amd64 3.6.2-3build3 [4,499 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libdcmtk-dev am

In [ ]:
from google.colab import files
uploaded = files.upload()

# `CR-MONO1-10-chest` is downloaded from: https://barre.dev/medical/samples/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from gradient_decode_dicom import decode_dicom_image, decode_dicom_data, tags
import tensorflow as tf
tf.enable_eager_execution()

uploaded_bytes = tf.constant(uploaded['CR-MONO1-10-chest'])
lossy_image = decode_dicom_image(uploaded_bytes, scale='auto', on_error='lossy', dtype=tf.uint8)
image = decode_dicom_image(uploaded_bytes, dtype=tf.uint16)
skipped = decode_dicom_image(uploaded_bytes, on_error='skip', dtype=tf.uint8)

fig, axes = plt.subplots(1,2, figsize=(10,10))
axes[0].imshow(np.squeeze(image.numpy()), cmap='gray')
axes[0].set_title('image')
axes[1].imshow(np.squeeze(lossy_image.numpy()), cmap='gray')
axes[1].set_title('lossy image');


In [ ]:
print(tf.rank(skipped))
print(skipped.numpy())

In [ ]:
tag_data = decode_dicom_data(uploaded_bytes, tags=[tags.PatientsName, 
                                                   tags.ImageType, 
                                                   tags.InstanceCreationDate])
print(tag_data.numpy())

# Gradient Decode Dicom Tensorflow Operation


## Getting started
1. Install the DCMTK toolkit 

  `sudo apt-get install libdcmtk-dev`
2. Install gradient_decode_dicom

  `pip3 install gradient-decode-dicom`
  

## Documentation
The `gradient_decode_dicom` has two operations, `decode_dicom_image` which decodes the pixel data from DICOM files, and `decode_dicom_data` which decodes tag information. `tags` contains useful DICOM tags such as `tags.PatientsName`. We borrow the same notation from the [`pydicom`](https://pydicom.github.io/) dicom package.

### Getting DICOM Image Data
```python
gradient_decode_dicom.decode_dicom_image(
    contents,
    color_dim=False,
    on_error='skip',
    scale='preserve'
    dtype=tf.uint16
    name=None
)
```

 - **`contents`**: A Tensor of type string. 0-D. The byte string encoded DICOM file
 - **`color_dim`**: An optional `bool`. Defaults to `False`. If `True`, a third channel will be appended to all images forming a 3-D tensor. A 1024 x 1024 grayscale image will be 1024 x 1024 x 1
 - **`on_error`**: Defaults to `skip`. This attribute establishes the behavior in case an error occurs on opening the image or if the output type cannot accomodate all the possible input values. For example if the user sets the output dtype to tf.uint8, but a dicom image stores a tf.uint16 type. `strict` throws an error. *`skip`* returns a 1-D empty tensor.  `lossy` continues with the operation scaling the value via the `scale` attribute. 
 - **`scale`**:  Defaults to `preserve`. This attribute establishes what to do with the scale of the input values. `auto` will autoscale the input values, if the output type is integer, `auto` will use the maximum output scale for example a `uint8` which stores values from [0, 255] can be linearly stretched to fill a `uint16` that is [0,65535]. If the output is float, `auto` will scale to [0,1]. `preserve` keeps the values as they are, an input value greater than the maximum possible output will be clipped. 
 - **`dtype`**: An optional `tf.DType` from: `tf.uint8, tf.uint16, tf.uint32, tf.uint64, tf.float16, tf.float32, tf.float64`. Defaults to `tf.uint16`. 
 - **`name`**: A name for the operation (optional).
 
 **Returns**

A `Tensor` of type `dtype` and the shape is determined by the DICOM file. 

 ### Getting DICOM Tag Data
 
 ```python
gradient_decode_dicom.decode_dicom_data(
    contents,
    tags= None, tf.uint32
    name=None
)
```

 - **`contents`**: A Tensor of type string. 0-D. The byte string encoded DICOM file
 - **`tags`**: A Tensor of type `tf.uint32` of any dimension. These `uint32` numbers map directly to DICOM tags
  - **`name`**: A name for the operation (optional).

**Returns**

A `Tensor` of type `tf.string` and same shape as `tags`.  If a dicom tag is a list of strings, they are combined into one string and seperated by a double backslash "`\\`". There is a bug in `dcmtk` if the tag is a list of numbers, only the zeroth element will be returned as a string.
